# Framing plan

In [62]:
# cadquery imports
import math
import numpy
import cadquery as cq
from jupyter_cadquery.cadquery import (Assembly, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar, set_defaults

set_defaults(axes=True, edge_accuracy=0.05, grid=True, axes0=True)

set_sidecar("Framing")
enable_replay()


Enabling jupyter_cadquery replay


In [85]:
class Wall:
  def __init__(self, width, height_left, height_right):
    self.width = width;
    self.height_left = height_left
    self.height_right = height_right
    self.board_long = 3.5/12
    self.board_short = 1.5/12
    self.spacing = 16/12

  def model(self):
    assembly = []
    angleRad = math.atan((self.height_left - self.height_right) / self.width)
    thicknessFromTopPlateAngle = self.board_short * (1.0/math.cos(angleRad) - 1)
    # baseplate
    assembly.append(cq.Workplane("XY").box(self.width, self.board_long, self.board_short, (False, False, False)))
    # top plate
    top_length = self.width/math.cos(angleRad)
    assembly.append(cq.Workplane("XY").box(top_length, self.board_long, self.board_short, (False, False, False))\
                    .rotate((0,0,0),(0,1,0),math.degrees(angleRad)).translate((0, 0, self.height_left - self.board_short - thicknessFromTopPlateAngle)))
    cutList = {"miter": math.degrees(angleRad), "base plate": self.width, "top plate (along both faces)": top_length}

    studIdx = 1
    # studs
    def stud(x):
      nonlocal studIdx
      height = self.height_left + x * (self.height_right - self.height_left) / self.width
      height = height - 2 * self.board_short - self.board_short * math.tan(angleRad) - thicknessFromTopPlateAngle
      cutList[f'stud {studIdx} (short side)'] = height
      studIdx = studIdx + 1
      return cq.Workplane("XY").box(self.board_short, self.board_long, height, (False, False, False)).translate((x, 0, self.board_short))
    for x in numpy.arange(0.0, self.width - 2 * self.board_short, self.spacing):
      assembly.append(stud(x))
    assembly.append(stud(self.width - self.board_short))
    print(cutList)

    ## calibration boards
    assembly.append(cq.Workplane("XY").box(self.width, self.board_long, self.board_short, (False, False, False))\
                    .translate((0,0,self.height_left-self.board_short)))
    assembly.append(cq.Workplane("XY").box(self.width, self.board_long, self.board_short, (False, False, False))\
                    .translate((0,0,self.height_right-self.board_short)))

    return Assembly(list(map(lambda p: Part(p), assembly)))

show(Wall(8, 8, 6).model())

{'miter': 14.036243467926479, 'base plate': 8, 'top plate (along both faces)': 8.246211251235321, 'stud 1 (short side)': 7.714902949199448, 'stud 2 (short side)': 7.381569615866115, 'stud 3 (short side)': 7.048236282532781, 'stud 4 (short side)': 6.714902949199448, 'stud 5 (short side)': 6.381569615866115, 'stud 6 (short side)': 6.048236282532782, 'stud 7 (short side)': 5.746152949199448}
Done, using side car 'Framing'


In [86]:
ft_side = 9.0
ft_corner = 4.5

ft_fence   = 6.0  # height along fence
ft_cornerH = 7.0  # height at hip break
ft_peak    = 8.0  # height at the peak

def pts =    [(0,0),    (ft_side,0), (ft_side,ft_corner), (ft_corner,ft_side), (0,ft_side), (0,0)   ]
def height = [ft_fence, ft_fence,    ft_cornerH,          ft_cornerH,          ft_fence,   ft_fence ]


walls = []
for i in range(0, len(pts) - 1):
    pt_left = pts[i]
    pt_right = pts[i + 1]  

SyntaxError: invalid syntax (<ipython-input-86-894f6f2e991c>, line 8)